# Import libraries

In [ ]:
import pymol
from pymol import cmd,stored
import numpy as np
import os
import glob
import subprocess
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
import MDAnalysis as mda
import seaborn as sns
import sys
import matplotlib.cm as cm
import unicodedata
from scipy.cluster.vq import kmeans2
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

# Set paths
This section takes .mol2 files from working directory (HOME) of your project and generates MD and docking topologies. .mol2 files with partial charges can be obtained from ORCA calculations 

In [2]:
NAME = "" # Name of the ligand to be docked
flex_name = "blind" # Name of the pocket for the flex docking. 'blind' - blind docking and etc.

HOME = "" # Full path to working directory
REPOSITORY = "" # Full path to the folder containing scripts
PATH_TO_DOCKING = f"{HOME}/{NAME}"
PATH_TO_MD = f"{HOME}/{NAME}"
PATH_TO_MOL2 = f"{HOME}/{NAME}"
SCRIPT_LIGAND = f"{REPOSITORY}/prep_wat.py"

SCRIPT_PARAM_LIGAND = f"{REPOSITORY}/prep_lig.py"

TARGET_PATH = PATH_TO_DOCKING

PATH_ADFR = f"XXX/ADFRsuite-1.0/bin" # Path to the ADFRSuite/bin package
PYTHONSH = f"{PATH_ADFR}/pythonsh" # Path to the ADFRSuite package


In [4]:
ligand_file = f"{NAME}_wat_charged.pdbqt" # Final name of the ligand .pdbqt file to be generated
receptor_file_pdb = "receptor.pdb" # Name of the receptor (protein) .pdb file

receptor_name = receptor_file_pdb.split('.')[0]
receptor_file = receptor_name+'.pdbqt' # Receptor .pdbqt file

grid_name = f"{flex_name}_site"
center_file = f"{flex_name}_center.xyz" #.xyz file containing the coordinates for the center of the search space

# dict of flexible receptor residues used for flexible docking in format "ARG114_ARG117_..."
residues_dict = { #
    'noflex':"",
    'site_name':"ARG114_ARG117_ASP183_ASP187_ARG186_VAL116_LEU115"
}
if flex_name in [""]: # list of sites where flexible docking will be used
    flex_residues = residues_dict[flex_name]
else:
    flex_residues = residues_dict['noflex']

# Preparation and docking

Copy all required files from a special directory.

The template directory contains all scripts necessary for docking.

In [8]:
TEMPLATE_PATH = f""
if not os.path.exists(TEMPLATE_PATH):
        print(f"Template directory does not exist.")
else:
    os.system(f"cp -r {TEMPLATE_PATH}/* {TARGET_PATH}/")

## Run parametrization of ligands

In [ ]:
os.chdir(PATH_TO_MOL2)
%run  {SCRIPT_PARAM_LIGAND} {NAME}.mol2 {PATH_TO_DOCKING} {PATH_TO_MD} {SCRIPT_LIGAND} {PATH_ADFR}

What you already should have in target folder:
1. Ligand file
2. Receptor file (pdb)
3. XYZ of the center of the docking box

Prepare receptor both normal and for flexible.

Check if your receptor is protonated. If not, you can use H++ server.

In [ ]:
! {PATH_ADFR}/prepare_receptor -r {receptor_file_pdb} -o {receptor_file} -v -A checkhydrogens
if (flex_residues != ""):
    ! {PYTHONSH} prepare_flexreceptor.py -r {receptor_file} -s {flex_residues} -v
    ! mv {receptor_name}_flex.pdbqt {flex_name}.pdbqt
    rigid_receptor_name = f"{receptor_file.split('.pdbqt')[0]}_rigid.pdbqt"

Prepare grid file

In [11]:
SIZE = 100 # Size of the docking box in Autodock units (default unit is 0.375 A)
os.chdir(PATH_TO_DOCKING)
# Optional: delete all files in the docking folder before running it again
clear_grid = False
if clear_grid:
    ! rm -rf {grid_name}*/*run*
    ! rm -rf {grid_name}*/*entity*
    ! rm -rf {grid_name}*/cluster*
    ! rm -rf {grid_name}*/*xml
    ! rm -rf {grid_name}*/*dlg

And run the Docking.

Simply copy printed commands and put them in the terminal in the main directory for docking of your ligand

Be careful with warnings about the number of evaluations!

In [ ]:
NRUNS = 2000 # Number of docking runs 1-8192
HEUR = 20_000_000 # Number of evaluations

if flex_residues != "":
    print(f"python3 gen_grids.py {PYTHONSH} {center_file} {rigid_receptor_name} {ligand_file} {SIZE} {grid_name}")
    print(f"cp {flex_name}.pdbqt {grid_name}1/")
    if(HEUR > 0):
        print(f"python3 run_adgpu_flex.py {rigid_receptor_name} {ligand_file} {NRUNS} 1 {flex_name}.pdbqt {grid_name} {HEUR}")
    else:
        print(f"python3 run_adgpu_flex.py {rigid_receptor_name} {ligand_file} {NRUNS} 1 {flex_name}.pdbqt {grid_name}")
else:
    print(f"python3 gen_grids.py {PYTHONSH} {center_file} {receptor_file} {ligand_file} {SIZE} {grid_name}")
    if HEUR >0:
        print(f"python3 run_adgpu.py {receptor_file} {ligand_file} {NRUNS} 1  {grid_name} {HEUR}")
    else:
        print(f"python3 run_adgpu.py {receptor_file} {ligand_file} {NRUNS} 1  {grid_name}")

# Analysis of results

## Parse results of docking 

In [ ]:
dataname=NAME
FOLDNAME = NAME

iffilter = True
n_filter = 7 # Threshold of pose population to be filtered
centers = glob.glob(f"{TARGET_PATH}/{grid_name}*")
i=0
print(centers)


k=0
fig = plt.figure(dpi=100)
fig_full = plt.figure(dpi=100)
pymol_load = [f'cd {grid_name}{i+1}; disable all; ' for i in range(Tot)]
for cent in centers:
	os.chdir(f"{TARGET_PATH}/{grid_name}{i+1}")
	energy = subprocess.getoutput("awk '/RANKING/ {if ($1==1 && $2==1) print $4;}' *.dlg")
	runs = glob.glob(f"{os.getcwd()}/*_run*")
	clusters = subprocess.getoutput("xmllint --xpath '//result/clustering_histogram/cluster' *.xml")
	# print(clusters)
	clusters = clusters.split("\n")
	print(clusters)
	cluster = np.empty(len(clusters),dtype = int)
	run_energy = np.empty(len(clusters),dtype = float)
	run = np.empty(len(clusters),dtype = int)
	nclust = np.empty(len(clusters),dtype = int)
	clustfile = np.empty(len(clusters),dtype = object)
	for idx, cl in enumerate(clusters):
		cl = cl.split()
		cluster[idx] = cl[1].split("=")[1].replace('"','')
		run_energy[idx] = cl[2].split("=")[1].replace('"','')
		run[idx] = cl[3].split("=")[1].replace('"','')
		nclust[idx] = cl[5].split("=")[1].replace("/>",'').replace('"','')
		clustfile[idx] = f"cluster{cluster[idx]}_{run_energy[idx]}_p{nclust[idx]}.pdbqt"
	clust_data = pd.DataFrame(data={'cluster':cluster,'energy':run_energy, 'nrun':run, 'n in cluster':nclust, 'clusterfile':clustfile})
	# clust_data=clust_data.astype(float)
	clust_data = clust_data.sort_values(by=['n in cluster'], ascending=False)
	if iffilter:
		clust_data_filter=clust_data.query(f'`n in cluster` >={n_filter}')
	
	ax = fig_full.add_subplot(1,1,0)
	clust_data.plot.bar(x='cluster', y='n in cluster', ax=ax, legend=False)
	ax.axvspan(0, len(clust_data_filter), color='orange', alpha=0.4,
	 label = f"{round(clust_data_filter['n in cluster'].sum()/clust_data['n in cluster'].sum()*100)} % Filtered for n > {n_filter}")
	plt.legend()
	rects = ax.patches
	labels = clust_data['energy']
	for rect, label in zip(rects, labels):
		height = rect.get_height()
		# ax.text(rect.get_x() + rect.get_width() / 2, height+0.01, label, ha='center', va='bottom')
	plt.title(f"{dataname}_grid{i+1}")

	ax2 = fig.add_subplot(1,1,0)
	clust_data_filter.plot.bar(x='cluster', y='n in cluster', ax=ax2, legend=False)
	
	rects = ax2.patches
	labels = clust_data_filter['energy']
	for rect, label in zip(rects, labels):
		height = rect.get_height()
		# ax2.text(rect.get_x() + rect.get_width() / 2, height+0.01, label, ha='center', va='bottom')
	
	plt.title(f"{dataname}_grid{i+1}")
	for file in clust_data_filter['clusterfile']:
		pymol_load[i]+=f"load {file}; "
	i+=1
	print(clust_data_filter["n in cluster"].sum())
fs=16
plt.xlabel("Cluster", fontsize=fs)
plt.ylabel("Population", fontsize=fs)
plt.tight_layout()
plt.show()

name_pymol = f"{flex_name}_{n_filter}_"
for idx,s in enumerate(pymol_load):
	pymol_load[idx]+=f" cd ..; group {name_pymol}{idx+1}, enabled"
print(len(clust_data_filter))
print("\n".join(pymol_load))
print("; ".join(pymol_load))

## Save chosen clusters as pdb files

In [7]:
clusters_folder = f"clusters_{n_filter}"
os.chdir(f"{TARGET_PATH}/{grid_name}1")
if not(os.path.exists(clusters_folder) and os.path.isdir(clusters_folder)):
    os.makedirs(clusters_folder)
for file in clust_data_filter['clusterfile']:
    # os.system(f"cp {file} {clusters_folder}/")
    os.system(f"obabel -ipdbqt {file} -opdb > {clusters_folder}/{file.split('.pdbqt')[0]}.pdb")

## Clusterize the results

In [ ]:
CLUSTER_PATH = f"{TARGET_PATH}/{grid_name}1/{clusters_folder}/"
N_INIT = 150


num = len(glob.glob(CLUSTER_PATH + "*.pdb"))
centers = np.empty([num,3],dtype=float)
population = np.empty([num],dtype=float)
# out_centers = np.array(["0"]*(num), dtype = object)
# out_clust = np.array(["0"]*(CLUSTER_NUM), dtype = object)
i=0
for file in glob.glob(CLUSTER_PATH + "*.pdb"):
    dock = mda.Universe(file)
    lig = dock.select_atoms("all")
    licent = str(lig.center_of_geometry()).replace('[','').replace(']','')
    licent = licent.encode()
    licent = licent.decode()
    licent = licent.split()
    licent =np.array(licent,dtype='float')
    centers[i,:] = licent
    population[i] = int(file.split('.pdb')[0].split('_p')[1])
    i+=1
print(CLUSTER_PATH)

In [9]:
import random
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import calinski_harabasz_score

sil_score = []
dbs_score = []
chs_score = []
gap_score = []
distorsions = []
if len(centers) > 40:
    SK = range(2,40)
else:
    SK = range(2,len(centers)-2)
rr = random.randrange(1,10000)
for i in SK:
    kmeans= KMeans(n_clusters=i,n_init=N_INIT, random_state=rr).fit(centers, sample_weight=population)
    labels= kmeans.labels_
    distorsions.append(kmeans.inertia_)
    sil = silhouette_score(centers,labels,metric="euclidean", random_state=rr)
    sil_score.append(sil)

In [10]:
import numpy.matlib
from kneed import KneeLocator

def sec_der(wcss):
    second_derivative = np.diff(wcss, 2)
    elbow_point = np.argmax(np.abs(second_derivative)) + 1  # +1 because np.diff reduces the length by 1
    return elbow_point
def elb_kneed(wcss):
    K = range(2,len(wcss)+2)
    kneedle = KneeLocator(K, wcss, curve='convex', direction='decreasing', S=1)
    elbow_point = kneedle.elbow
    return elbow_point

In [ ]:
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
fig = plt.figure(dpi=300)
plt.subplot(2,1,1)
plt.plot(SK, distorsions)

elbow_point = elb_kneed(distorsions) + 1  # +1 because index starts from 0

print(f'Optimal number of clusters: {elbow_point}')
plt.axvline(x=elbow_point, linestyle='--', color='red', label = f"Elbow at {elbow_point}")
plt.legend()
plt.grid(True)
plt.title('Elbow curve')

plt.subplot(2,1,2)
sil_centers = pd.DataFrame({'Clusters' : SK, 'Sil Score' : sil_score})
sns.lineplot(x = 'Clusters', y = 'Sil Score', data = sil_centers, marker="+")
plt.title('Sillhoutte')
sil_max = np.argmax(np.abs(sil_score)) + 2
print(sil_score[sil_max])
plt.axvline(x=elbow_point, linestyle='--', color='red', label = f"Elbow at {elbow_point}")
plt.axvline(x=sil_max, linestyle='--', color='purple', label = f"Max at {sil_max}")
plt.legend()


plt.tight_layout()
plt.suptitle(f"{FOLDNAME}/{grid_name}")
plt.show()

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
max_d = 65  # This is the distance threshold
Z = linkage(centers, method='ward', metric='euclidean')
clusters_cut = fcluster(Z, max_d, criterion='distance')
plt.figure(dpi=300)
dendrogram(Z)
plt.axhline(y=max_d,linestyle='--', color='red', label = f"Cutoff {max_d}, {max(clusters_cut)} clusters")
plt.title(f"{FOLDNAME}/{grid_name} Dendrogram")
plt.xlabel('Sample index')
plt.ylabel('Distance')
plt.legend()
print(max(clusters_cut))
# Z_cut = linkage(clusters_cut, method='ward')
# dendrogram(Z_cut)
plt.show()

# Generate clusters

Throw a d20 dice and choose a number of clusters

In [15]:
NUM_CLUSTERS = 12
kmeans = KMeans(n_clusters=NUM_CLUSTERS, n_init=N_INIT).fit(centers, sample_weight = population)
labels = kmeans.predict(centers)
centroids = kmeans.cluster_centers_
out_centers = np.array(["0"]*(num), dtype = object)
out_clust = np.array(["0"]*(NUM_CLUSTERS*3), dtype = object)
weights = np.zeros([num],dtype=float)

for j,l in enumerate(labels):
    weights[l] += population[j]
i=0

combined = list(zip(centroids, weights))
# Sort based on the weights
sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)
# Unzip the sorted arrays
sorted_centroids, sorted_weights = zip(*sorted_combined)

for clust in sorted_centroids:
    cen = str(clust).replace('[','').replace(']','')
    cen = "F    "+ cen
    out_clust[i] = f"1"
    out_clust[i+1] = f"KM{i//3}_{NAME}_p{round(sorted_weights[i//3])}"
    out_clust[i+2] = cen
    i+=3


out_centers = np.insert(out_centers, 0,f"Coordinates of DiffDock pockets for {NAME}")
# out_clust = np.insert(out_clust, 0,f"Clusterized {CLUSTER_NUM} pockets for {NAME}")
np.savetxt(CLUSTER_PATH + f'{NAME}_centers.xyz',out_centers,fmt="%s", encoding = 'latin1')
np.savetxt(CLUSTER_PATH + f'{NAME}_clusters_{NUM_CLUSTERS}.xyz',out_clust,fmt="%s", encoding = 'latin1')


## Export results to PyMol

In [ ]:
def labelClust(selname, divider):
    divider = str(divider)
    seleobj = cmd.get_object_list(selname)
    for obj in seleobj:
        # print(obj, divider)
        name = f"{obj.split(divider)[-1]} "
        if divider[-1] == 'p': # for population to easy
            cmd.label(obj,f'"p{name}"')
        else:
            cmd.label(obj,f'"{name}"')
            
def renameObj_cluster(nameSelection = "pk1", spinlabel="C34R1"):
    seleobjs = cmd.get_object_list(nameSelection)
    for obj in seleobjs:
        labelobj = cmd.get_object_list(spinlabel)
        labelname = ""
        statenum = cmd.count_states(spinlabel)
        dist=np.zeros(statenum)
        for i in range(statenum):
                dist[i]+=(cmd.get_distance(atom1=f"{obj}", atom2=f"{spinlabel} and name O1", state = i))
        avdist = np.mean(dist)
        disp = np.std(dist)
        # print("Average distance is ", avdist)
        # print("Dispersion is ", disp)
        if(obj[-1] !='A'):
            cmd.set_name(obj, f"{obj}_{round(avdist)}A")
cmd.extend("renameObj_cluster", renameObj_cluster)

def getCluster_params(selname, divider):
    data = []
    divider = str(divider)
    seleobj = cmd.get_object_list(selname)
    for obj in seleobj:
        print(obj, divider)
        name = f"{obj.split(divider)[-1]} "
        if divider[-1] == 'p': # for population to easy
            name = f"p{name}"
        clust_num = obj.split('_')[0].split('KM')[-1]
        pop = int(name.split('_')[0][1:])
        dist = int(name.split('_')[1][:-2])
        data.append({'Cluster Population': pop, 'Average Distance (A)': dist})
    df = pd.DataFrame(data)
    # Rename the index
    df.index.name = "Cluster Number"
    df.index +=1
    df = df.T
    return df
        

In [ ]:
pymol_name = f"{NAME}_docking.pse"
os.chdir(PATH_TO_DOCKING)
if not os.path.isfile(pymol_name):
    pymol_template = "apo_template.pse"
    cmd.load(pymol_template)
    cmd.load(CLUSTER_PATH + f'{NAME}_clusters.xyz')
    cmd.split_states(f"{NAME}_clusters")
    cmd.delete(f"{NAME}_clusters")
    cmd.group(f"{NAME}_clusters", "KM*")
    for file in clust_data_filter['clusterfile']:
        cmd.load(f"{TARGET_PATH}/{grid_name}1/" + file)
    cmd.remove("name WAT")
    cmd.group("poses", f"cluster*_*")
    renameObj_cluster("KM*")
    labelClust("KM*", "_p")
    table = getCluster_params("KM*", "_p")

    cmd.zoom(f"receptor")

    cmd.save(pymol_name)
else:
    cmd.load(pymol_name)
    table = getCluster_params("KM*", "_p")


Show Dataframe with information about clusters

In [ ]:
print(f"renameObj_cluster KM*")
print(f"labelClust KM*, _p")
# if flex_residues != "":
table
